# _PREPARATION_

In [ ]:
!conda list

In [ ]:
!pip install cvzone

# _CODE TESTING_

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import time

In [ ]:
# Inisialisasi video capture
video_path = "D:/NWR27/MY_FILES/Videos/CCTV/New/Video Crop v2.mp4"
cap = cv2.VideoCapture(video_path)

# Inisialisasi YOLO model
model_path = "D:/NWR27/AI_CCTV/.runs/detect/employees_3_original++/weights/best.pt"
model = YOLO(model_path)

# Nama kelas
classNames = ["Edo", "Asep", "Rose", "Cica", "Lilis", "Rohani", "Fitri"]

# Dimensi untuk imshow
scaleof = 0.75  # 0 to 1.5 (1280 x 720 default video resolution)
newDim = (int(1280 * scaleof), int(720 * scaleof))

In [ ]:
# Mendapatkan lebar, tinggi, dan frame rate asli video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / original_fps)  # Delay dalam milidetik

# Inisialisasi VideoWriter object dengan frame rate asli
output_path = ".runs/videos/output_video.avi"
out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*"XVID"),
    original_fps,
    (frame_width, frame_height),
)

In [ ]:
while True:
    start_time = time.time()
    success, img = cap.read()
    if not success:
        print("...Gagal memuat frame video...")
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Nama Kelas
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            if conf > 0:
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cvzone.putTextRect(
                    img,
                    f"{currentClass} {conf}",
                    (max(0, x1), max(35, y1)),
                    scale=2,
                    thickness=2,
                    colorT=(0, 0, 255),
                    colorR=(0, 255, 255),
                    colorB=(0, 252, 0),
                    offset=5,
                )

    # Menulis frame ke file video
    out.write(img)

    # Menampilkan frame
    img_resized = cv2.resize(img, newDim)
    cv2.imshow("Image", img_resized)

    # Menghitung waktu pemrosesan dan menambahkan delay
    processing_time = time.time() - start_time
    wait_time = max(1, frame_delay - int(processing_time * 1000))  # Pastikan waktu tunggu tidak negatif
    if cv2.waitKey(wait_time) & 0xFF == ord("q"):
        break

# Melepas objek VideoCapture dan VideoWriter
cap.release()
out.release()
cv2.destroyAllWindows()